<a href="https://colab.research.google.com/github/vkrisvasan/llamaKV/blob/main/llamaindexYouTubekv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install youtube-transcript-api
!pip install llama-index-readers-youtube-transcript
!pip install llama-index llama-index-llms-groq groq llama-index-embeddings-huggingface

In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq

In [3]:
import os
import getpass
credential_names = ["GROQ_API_KEY","HF_TOKEN"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

Provide your...GROQ_API_KEY··········
Provide your...HF_TOKEN··········


In [ ]:
HF_TOKEN=os.environ["HF_TOKEN"]

In [4]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(
    ytlinks=["https://www.youtube.com/watch?v=i3OYlaoj-BM"]
)

In [5]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
print(f"Loaded {len(documents)} documents")
print(f"Split into {len(nodes)} nodes")
print(f"nodes [0] {nodes[0].metadata} ")

Loaded 1 documents
Split into 8 nodes
nodes [0] {'video_id': 'i3OYlaoj-BM'} 


In [7]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
llm = Groq(model="llama-3.1-8b-instant", api_key=os.environ["GROQ_API_KEY"])

In [9]:
from llama_index.core import Settings
from llama_index.llms.groq import Groq

Settings.llm = Groq(model="llama-3.1-8b-instant",api_key=os.environ["GROQ_API_KEY"])

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [10]:
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, node_parser=nodes)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/8 [00:00<?, ?it/s]

In [11]:
#service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

<ipython-input-11-ab72642cc4df>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)


In [12]:
#vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/8 [00:00<?, ?it/s]

In [13]:
vector_index.storage_context.persist(persist_dir="./storage_mini")
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")
index = load_index_from_storage(storage_context, service_context=service_context)
query_engine = index.as_query_engine(service_context=service_context)




In [14]:
query = "summarise the document"
resp = query_engine.query(query)
print(resp)

The document discusses the use of the CLIP model for various computer vision tasks, including image classification, object localization, and object detection. The author explains how CLIP can be used to compare image embeddings with class label embeddings to make predictions, and how this can be applied to object localization by breaking down the image into patches and calculating similarity scores for each patch. The author also mentions an alternative approach using the occlusion algorithm, where a black patch is slid over the image to hide parts of it and calculate similarity scores. The document concludes by outlining the steps to implement these tasks using a small demo dataset.


In [15]:
# prompt: print the text in a paragraph format

for doc in documents:
  print(doc.text)


the imaginative laws scale visual
recognition challenge was a
world-changing competition
that ran from around 2010 to 2017.
during his time the competition acted as
the place to go if you needed to find
what the current state of the art was in
image classification object localization
object detection as well as that
2012 onwards it really acted as the
Catalyst of the explosion in deep
learning researchers fine-tuned better
performing computer vision models year
on year but there was a unquestioned
assumption causing problems
it was assumed that every new task
required model fine tuning this required
a lot of data and a lot of data required
a lot of capital and time it wasn't
until recently that this assumption was
challenged and proven wrong the
astonishing rise of what are called
multi-modal models has made the what was
thought impossible very possible across
various domains and tasks one of those
is called zero shot object detection and
localization now zero shot refers to
taking a m